# PandasAI v3: The Semantic Layer

This notebook covers the semantic layer in PandasAI, a powerful feature for defining data schemas and transformations.

## 1. Creating a Semantic Layer from a CSV

In [ ]:
import pandas as pd
import pandasai as pai
from pandasai_litellm.litellm import LiteLLM

# Configure LLM
llm = LiteLLM(model="gpt-5-mini", api_key="YOUR_OPENAI_API_KEY")
pai.config.set({"llm": llm})

# Create a sample DataFrame and save to CSV
data = {
    'transaction_id': [1, 2, 3, 4, 5],
    'sale_date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
    'quantity': [10, 12, 8, 15, 7],
    'price': [1.2, 1.5, 1.0, 1.8, 1.3],
    'is_online': [True, False, True, False, True]
}
df = pd.DataFrame(data)
df.to_csv('sales_data.csv', index=False)

# Load data and create semantic layer
file = pai.read_csv('sales_data.csv')

sales_df = pai.create(
    path="my-org/sales-data",
    df=file,
    description="Sales data from our retail stores",
    columns=[
        {"name": "transaction_id", "type": "string", "description": "Unique identifier for each sale"},
        {"name": "sale_date", "type": "datetime", "description": "Date and time of the sale"},
        {"name": "quantity", "type": "integer", "description": "Number of units sold"},
        {"name": "price", "type": "float", "description": "Price per unit in USD"},
        {"name": "is_online", "type": "boolean", "description": "Whether the sale was made online"}
    ]
)

response = sales_df.chat("What is the total revenue?")
print(response)

## 2. Creating a Semantic Layer from a SQL Database

In [ ]:
!pip install pandasai-sql

In [ ]:
# This is a conceptual example. You would need a running SQL database.
# Replace with your actual database credentials.
import os

os.environ['DB_HOST'] = 'localhost'
os.environ['DB_USER'] = 'user'
os.environ['DB_PASSWORD'] = 'password'
os.environ['DB_NAME'] = 'database'

sql_table = pai.create(
    path="company/health-data",
    description="Heart disease dataset from MySQL database",
    source={
        "type": "mysql",
        "connection": {
            "host": "${DB_HOST}",
            "port": 3306,
            "user": "${DB_USER}",
            "password": "${DB_PASSWORD}",
            "database": "${DB_NAME}"
        },
        "table": "heart_data"
    }
)

# response = sql_table.chat("What is the average age of patients with heart disease?")
# print(response)

## 3. Using YAML to Define a Semantic Schema

In [ ]:
!mkdir -p datasets/my-org/sales-data

In [ ]:
df.to_csv('datasets/my-org/sales-data/data.csv', index=False)

In [ ]:
yaml_schema = """
description: Daily sales transactions from all retail stores.
columns:
  - name: transaction_id
    type: string
    description: Unique identifier for each sale
  - name: sale_date
    type: datetime
    description: Date and time of the sale
"""

with open('datasets/my-org/sales-data/schema.yaml', 'w') as f:
    f.write(yaml_schema)

In [ ]:
# Load the dataset using the YAML schema
sales_data_from_yaml = pai.load("my-org/sales-data")
response = sales_data_from_yaml.chat("What is the total revenue?")
print(response)

## 4. Transformations and Column Expressions

In [ ]:
yaml_schema_with_transformations = """
description: Daily sales transactions from all retail stores.
columns:
  - name: transaction_id
    type: string
    description: Unique identifier for each sale
  - name: total_revenue
    type: float
    description: Total revenue including tax
    expression: "quantity * price * 1.08" # Assuming 8% tax
transformations:
  - type: anonymize
    params:
      columns:
        - transaction_id
      method: hash
"""

with open('datasets/my-org/sales-data/schema.yaml', 'w') as f:
    f.write(yaml_schema_with_transformations)